[PyTorch之torchvision.transforms详解](https://blog.csdn.net/qq_37555071/article/details/107532319)
正如其名所见，transforms是一个用于对图片进行变换的工具包，例如裁剪、缩放、垂直翻转、水平翻转等等，这些操作具有单独的其详细的定义(见链接博客)，而如果需要对图像进行一系列的操作也即操作组合，可以使用Compose()传入的是一个操作列表

In [29]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader

## 1. 导入数据
trans = transforms.ToTensor() # 转为张量
minist_train = FashionMNIST(root="../../data", train=True, transform=trans)
minist_test = FashionMNIST(root="../../data", train=False, transform=trans)
## 这是小批量读取
batch_size = 256
train_iter = DataLoader(minist_train, batch_size, shuffle=True, num_workers=4)
test_iter = DataLoader(minist_test, batch_size, shuffle=False, num_workers=4)
## 测试数据读取是否成功
X0, y0 = next(iter(train_iter))
X0.shape, y0.shape  # 说明每一个batch包含的是256张图片及其标记，每个图片大小为28x28

(torch.Size([256, 1, 28, 28]), torch.Size([256]))

In [30]:
## 此处为实验楼
X0r01 = X0.reshape(batch_size, -1)[0:2, :]
W0r01 = torch.normal(0, 0.1, size=(784, 10))
b0r01 = torch.normal(0, 0.01, size=(1, 10))
XW0r01 = torch.matmul(X0r01, W0r01)
expXWb = torch.exp(XW0r01+b0r01)
eXWbsum = expXWb.sum(dim=1, keepdim=True)
pr = expXWb/eXWbsum
lby0r01 = torch.tensor([3, 4])
pr[[0,1],lby0r01]

tensor([0.0606, 0.1267])

In [40]:
num_inputs = 28*28
num_outputs = 10
W = torch.normal(0, 0.1, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)
## W:784x10, X:256x1x28x28, b: 1x10, y 1x256
## 2. 编写假设函数softmax X --> h(X)
## (XW+b)[256x10], 对于每一行求exp/sum(exp)
def softmax(X):
    # 256x784, 注意最后一个batch可能不是256
    expXWb = torch.exp(torch.matmul(X.reshape(-1, num_inputs), W)+b)
    sumXWb = expXWb.sum(dim=1, keepdim=True)
    return expXWb / sumXWb
## 3. 编写损失函数
def lossCrossEntropy(y_hat, y):
    lCEbatch = -torch.log(y_hat[range(len(y)), y])
    return lCEbatch.sum()
## 4. 编写优化函数
def gradientDescent(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()
## 5. 进行训练
epoch_num = 10
lr = 0.1
for epoch in range(epoch_num):
    for X, y in train_iter:
        l = lossCrossEntropy(softmax(X), y)
        l.backward()
        gradientDescent([W, b], lr, batch_size)
    print(f"train loss = {float(l):.5f}")

train loss = 42.36957
train loss = 41.88358
train loss = 35.38435
train loss = 50.34916
train loss = 45.52011
train loss = 51.93243
train loss = 48.97696
train loss = 44.46736
train loss = 39.56340
train loss = 35.18392


In [42]:
# 计算在测试集上的精度
def cmpY(y_hat, y):
    return (y == y_hat.argmax(dim=1)).sum()
acc_cnt = 0
sam_tot = 0
for X, y in test_iter:
    acc_cnt += cmpY(softmax(X), y)
    sam_tot += len(y)
acc_cnt, sam_tot, acc_cnt/sam_tot

(tensor(8319), 10000, tensor(0.8319))